<a href="https://colab.research.google.com/github/RahaviSelvarajan/yolov5_colab/blob/main/SBI_YOLO_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!python --version

##***Mount Google Drive***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##***Mount Onedrive to get the dataset***

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

**Run the below cell only when onedrive gets disconnected in the middle of training and mount it again using the above command.**

In [ ]:
!fusermount -u /content/onedrive

##***Git Clone YOLOv5-Pytorch model***

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12388, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12388 (delta 1), reused 4 (delta 0), pack-reused 12380
Receiving objects: 100% (12388/12388), 12.18 MiB | 33.25 MiB/s, done.
Resolving deltas: 100% (8544/8544), done.


Install the required python packages

In [ ]:
%cd yolov5
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 30.6 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Copy the data configuration to the data folder

In [ ]:
# Copy data configuration file into yolo directory
!cp /content/drive/MyDrive/Yolov5/sleepaid.yaml /content/yolov5/data/

##**Training**

In [ ]:
# Import libraries
import os
import glob
import shutil
from pathlib import Path

In [ ]:
# Get the train and test dataset from different folds
def train_test_split(root_dir, test_fold):

    # Make directory for training and testing images
    train_imgs = Path('/content/yolov5/data/images/train')
    test_imgs = Path('/content/yolov5/data/images/test')

    if train_imgs.exists():
      shutil.rmtree(train_imgs)
    if test_imgs.exists():
      shutil.rmtree(test_imgs)

    os.mkdir(train_imgs)
    os.mkdir(test_imgs)

    # Make directory for target labels
    train_labels = Path('/content/yolov5/data/labels/train')
    test_labels = Path('/content/yolov5/data/labels/test')

    if train_labels.exists():
      shutil.rmtree(train_labels)
    if test_labels.exists():
      shutil.rmtree(test_labels)

    os.makedirs(train_labels)
    os.makedirs(test_labels)

    for j, image in enumerate(glob.iglob(os.path.join('/content/drive/MyDrive/Negatives', "*.jpg"))):
      if j<150:
        shutil.copy(image, train_imgs)
      else:
        shutil.copy(image, test_imgs)

    for i, folder in enumerate(os.listdir(root_dir)):
      folder_path = os.path.join(root_dir, folder)
      file_list = [image for image in glob.glob(os.path.join(folder_path, '*.jpg'))]

      # Get the train folds
      if i != test_fold:
        for image in file_list:
          shutil.copy(image, train_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', train_labels)

      # Get the test fold
      elif i == test_fold:
        for image in file_list:
          shutil.copy(image, test_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', test_labels)

for i in range(1):
  train_test_split('/content/drive/MyDrive/Dataset', i)
  !python train.py --batch-size 8 --img-size 1920 --data 'sleepaid.yaml' --weights 'yolov5s.pt' --epochs 300 --cfg 'yolov5s.yaml'  \
                   --save-period 50 --patience 20 --hyp '/content/drive/MyDrive/Yolov5/hyp.yaml' --project '/content/drive/MyDrive/Yolov5/runs/final_train' 
  !mv /content/drive/MyDrive/Yolov5/runs/final_train/exp /content/drive/MyDrive/Yolov5/runs/final_train/results"$i"

##**Validation**

In [ ]:
!unzip /content/drive/MyDrive/val.zip

In [ ]:
# unzip the DWU128 participant
!unzip /content/drive/MyDrive/DWU128.zip -d /content/DWU128

In [ ]:
val_imgs = Path('/content/yolov5/data/images/test')
val_labels = Path('/content/yolov5/data/labels/test')

if val_imgs.exists():
  shutil.rmtree(val_imgs)
if val_labels.exists():
  shutil.rmtree(val_labels)

os.mkdir(val_imgs)
os.makedirs(val_labels)  

# for folder in os.listdir('/content/DWU128'):
#   folder_path = os.path.join('/content/DWU128', folder)
#   if not folder.startswith('.'):
for file in os.listdir('/content/DWU128'):
  extension = os.path.splitext(file)[1]
  f_path = os.path.join('/content/DWU128', file)
  if extension == '.txt':
    shutil.copy(f_path, val_labels)
  elif extension == '.jpg':
    shutil.copy(f_path, val_imgs)

In [ ]:
!python val.py --batch-size 8 --img-size 1920 --data 'sleepaid.yaml' --weights '/content/drive/MyDrive/Yolov5/runs/final_train/results2/weights/best.pt' --project '/content/drive/MyDrive/Yolov5/runs/final_train' 

##**Testing**

In [ ]:
!python detect.py --source '/content/drive/MyDrive/B-DWU128.mp4' --max-det 1 --conf-thres 0.1 --iou-thres 0.2 --img-size 1920 --weights '/content/drive/MyDrive/Yolov5/runs/final_train/results2/weights/best.pt' --save-txt --data '/content/yolov5/data/sleepaid.yaml'

##**Position Time Quantification**

In [ ]:
import cv2
import torch
from datetime import datetime
import pandas as pd

In [ ]:
def load_model():

    model = torch.hub.load('ultralytics/yolov5',
                            'custom', path='/content/drive/MyDrive/Yolov5/runs/final_train/results2/weights/best.pt')
    
    model.max_det = 1
    model.conf_thres = 0.1
    model.iou_thres = 0.2
    
    classes = ['B-Left_recovery', 'B-Left_lateral', 'B-Left_tilt', 'B-Supine', 'B-Supine_thorax_with_Left_pelvic_tilt', 
                'B-Supine_thorax_with_Right_pelvic_tilt', 'B-Supine_pelvis_with_Left_thorax_tilt', 
                'B-Supine_pelvis_with_Right_thorax_tilt', 'B-Right_tilt', 'B-Right_lateral', 'B-Right_recovery', 'B-Sitting', 
                'N_B-Left_recovery', 'N_B-Left_lateral', 'N_B-Left_tilt', 'N_B-Supine', 'N_B-Supine_thorax_with_Left_pelvic_tilt', 
                'N_B-Supine_thorax_with_Right_pelvic_tilt', 'N_B-Supine_pelvis_with_Left_thorax_tilt', 
                'N_B-Supine_pelvis_with_Right_thorax_tilt', 'N_B-Right_tilt', 'N_B-Right_lateral', 'N_B-Right_recovery', 'N_B-Sitting']
                    
    return model, classes

    
def detect_objects(video_file, model, classes):

  count = 0 

  cap = cv2.VideoCapture(video_file)
  fps = cap.get(cv2.CAP_PROP_FPS)
  objects = list()
  pt_data = dict()
  while True:
      ret, frame = cap.read()
      if ret:
        result = model(frame)
        labels = result.xyxyn[0][:, -1].cpu().numpy()
        if len(labels) != 0:
          count += 1
          objects.append(classes[int(labels[0])])
        # if len(objects) > 2:
        #   if objects[-1] != objects[-2]:      
        #     if objects[-2] not in pt_data:
        #       pt_data[objects[-2]] = 0
        #     pt_data[objects[-2]] += count/fps     
        #     count = 0
      else:
          return objects

In [ ]:
model, classes = load_model()

data = detect_objects('/content/drive/MyDrive/B-DWU128.mp4', model, classes)

print(data)

In [ ]:
from collections import Counter
counts = Counter(data)
print(counts)

In [ ]:
df = pd.DataFrame.from_dict(data, orient='index', columns=['Time spent'])
# df.index.name = 'Position'
display(df)

,Time spent
B-Left_lateral,10.7
B-Supine,15.1
B-Supine_pelvis_with_Right_thorax_tilt,0.1
B-Right_recovery,0.1


In [ ]:
nb_data = detect_objects('/content/drive/MyDrive/N_B-DWU128.mp4', model, classes)

In [ ]:
df = pd.DataFrame.from_dict(nb_data, orient='index', columns=['Time spent'])
# df.index.name = 'Position'
display(df)

,Time spent
N_B-Left_recovery,0.6
N_B-Left_lateral,5.0
N_B-Supine,21.9
N_B-Supine_thorax_with_Left_pelvic_tilt,5.1
N_B-Left_tilt,0.6
N_B-Supine_thorax_with_Right_pelvic_tilt,6.2
N_B-Supine_pelvis_with_Left_thorax_tilt,11.7
